In [2]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import batch_normalization
import os
import seaborn as sns

2023-04-05 13:28:04.877173: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Set up folder
p = os.path.abspath('.')
benchmark_folder = os.path.join(p, f'Benchmark_data')
if not os.path.exists(benchmark_folder):
    os.mkdir(benchmark_folder)
    print(f"{benchmark_folder} created")
else:
    print(f"{benchmark_folder} already exists")
output_dir = benchmark_folder

/home/art_usr/source_code/CFU_counter/Benchmark_data already exists


In [10]:
# Read filenames in the raw data folder
image_names = []
labels = []
raw_folder = os.path.join(p, f'Thresholded')

for file in glob.glob(f'{raw_folder}/*.jpg'):
	filename = file.split('/')[-1]
	num_colonies = int(filename.split('-')[2].strip().split('.')[0])
	if num_colonies == 0:
		labels.append(0)
	else:
		labels.append(1)
	image_names.append(filename)

image_names = np.array(image_names)
labels = np.array(labels)
print(f"Im shape {image_names.shape}")

Im shape (4352,)


In [11]:
#  Get two arrays with filenames for positive and negative images
pos_im_files = [image_names[i] for i in range(image_names.shape[0]) if labels[i]]
pos_im_files = np.array(pos_im_files)
neg_im_files = [image_names[i] for i in range(image_names.shape[0]) if not labels[i]]
neg_im_files = np.array(neg_im_files)
num_pos = pos_im_files.shape[0]
num_neg = neg_im_files.shape[0]
print(f'There are {num_pos} images with bacteria and {num_neg} without')

There are 366 images with bacteria and 3986 without


In [15]:
#  Make a represantive benchmark data set with ~50 positive images 

#  Make array of random indeces for negative images
np.random.seed(0) # For reproducibility
idx = np.random.choice(np.arange(neg_im_files.shape[0]), int(0.14*num_neg), replace = False)
#  Apply these indecies to the array of negativeimage names
neg_selected_files = neg_im_files[idx]
#  Move the resulting files to a benchmark folder
for file in neg_selected_files:
    if os.path.exists(os.path.join(raw_folder, file)):
        os.rename(os.path.join(raw_folder, file), os.path.join(benchmark_folder, file))

#  repeat the same for positive images

#  Make array of random indeces for negative images
idx = np.random.choice(np.arange(pos_im_files.shape[0]), int(0.14*num_pos), replace = False)
pos_selected_files = pos_im_files[idx]
for file in pos_selected_files:
    if os.path.exists(os.path.join(raw_folder, file)):
        os.rename(os.path.join(raw_folder, file), os.path.join(benchmark_folder, file))